In [73]:
import json
import random
import string

def generate_data():
    key_length = 10
    value_length = 20
    key = ''.join(random.choices(string.ascii_uppercase + string.digits, k=key_length))
    value = ''.join(random.choices(string.digits, k=value_length))
    return key, value

data = {}
num_entries = 1000000  # Количество записей

for _ in range(num_entries):
    key, value = generate_data()
    data[key] = value

file_name = "redis_data.json"

with open(file_name, "w") as json_file:
    json.dump(data, json_file)

In [74]:
import redis
import json

In [75]:
r = redis.Redis(host='127.0.0.1', port=6379)

In [76]:
%%time
with open("redis_data.json") as f:
    large_json = json.loads(f.read())

CPU times: user 331 ms, sys: 78.7 ms, total: 410 ms
Wall time: 415 ms


In [77]:
%%time
# Строка
r.set('large_json_string', json.dumps(large_json))

CPU times: user 463 ms, sys: 103 ms, total: 566 ms
Wall time: 728 ms


True

In [78]:
%%time
string_data = r.get('large_json_string')

CPU times: user 12 ms, sys: 48.6 ms, total: 60.5 ms
Wall time: 178 ms


In [79]:
%%time
# Хэш-набор
r.hset('large_json_hash', mapping=large_json)

CPU times: user 1.59 s, sys: 86.9 ms, total: 1.68 s
Wall time: 2.17 s


1000000

In [80]:
%%time
hash_data = r.hgetall('large_json_hash')

CPU times: user 1.82 s, sys: 133 ms, total: 1.95 s
Wall time: 2.19 s


In [81]:
%%time
# Упорядоченное множество
r.zadd('large_json_zset', large_json)


CPU times: user 1.61 s, sys: 65.9 ms, total: 1.68 s
Wall time: 3.41 s


1000000

In [82]:
%%time
zset_data = r.zrange('large_json_zset', 0, -1)

CPU times: user 876 ms, sys: 25.9 ms, total: 902 ms
Wall time: 1.13 s


In [83]:
%%time

# Список
for k, v in large_json.items():
    r.lpush("large_json_list", v)

CPU times: user 23.4 s, sys: 11.7 s, total: 35.1 s
Wall time: 3min 26s


In [84]:
%%time
list_data = r.lrange('large_json_list', 0, -1)

CPU times: user 866 ms, sys: 56 ms, total: 922 ms
Wall time: 990 ms
